In [1]:
# https://github.com/knowsuchagency/jupyter-mypy/blob/master/main.py
"""
Add mypy type-checking cell magic to jupyter/ipython.
Save this script to your ipython profile's startup directory.
IPython's directories can be found via `ipython locate [profile]` to find the current ipython directory and ipython profile directory, respectively.
For example, this file could exist on a path like this on mac:
/Users/yourusername/.ipython/profile_default/startup/typecheck.py
where /Users/yourusername/.ipython/profile_default/ is the ipython directory for
the default profile.
The line magic is called "typecheck" to avoid namespace conflict with the mypy
package.
"""

from IPython.core.magic import register_cell_magic


@register_cell_magic
def typecheck(line, cell):
    """
    Run the following cell though mypy.
    Any parameters that would normally be passed to the mypy cli
    can be passed on the first line, with the exception of the
    -c flag we use to pass the code from the cell we want to execute
     i.e.
    %%typecheck --ignore-missing-imports
    ...
    ...
    ...
    mypy stdout and stderr will print prior to output of cell. If there are no conflicts,
    nothing will be printed by mypy.
    """

    from IPython import get_ipython
    from mypy import api
    
    cell = '\n' + cell
    mypy_result = api.run(line.split() + ['-c', cell])

    if mypy_result[0]:  # print mypy stdout
        print(mypy_result[0])

    if mypy_result[1]:  # print mypy stderr
        print(mypy_result[1])

    shell = get_ipython()
    shell.run_cell(cell)
    
def reveal_type(*args, **argkw):
    pass



# Function Annotations

In [15]:
import math

def circumference(radius: float) -> float:
    return 2 * math.pi * radius

In [16]:
circumference(1.23)

7.728317927830891

In [17]:
print(circumference.__annotations__)

{'radius': <class 'float'>, 'return': <class 'float'>}


# Variable Annotations

In [5]:
pi: float = 3.142

def circumference(radius: float) -> float:
    return 2 * pi * radius

In [6]:
circumference(1)

6.284

In [8]:
print(__annotations__)

{'pi': <class 'float'>}


In [10]:
nothing: str

In [11]:
nothing

NameError: name 'nothing' is not defined

In [13]:
print(__annotations__)

{'pi': <class 'float'>, 'nothing': <class 'str'>}


# Type Comments

In [20]:
import math

def circumference_v2(radius):
    # type: (float) -> float
    return 2 * math.pi * radius

In [21]:
circumference_v2.__annotations__

{}

In [22]:
def headline(text, width=80, fill_char="-"):
    # type: (str, int, str) -> str
    return f" {text.title()} ".center(width, fill_char)

print(headline("type comments work", width=40))

---------- Type Comments Work ----------


In [8]:
%%typecheck

# choose.py

import random
from typing import Any, Sequence

def choose(items: Sequence[Any]) -> Any:
    return random.choice(items)

names = ["Guido", "Jukka", "Ivan"]
reveal_type(names)

name = choose(names)
reveal_type(name)

<string>:11: error: Revealed type is 'builtins.list[builtins.str*]'
<string>:14: error: Revealed type is 'Any'



# Playing With Python Types, Part 2

## Type Variables

In [20]:
%%typecheck
# choose.py

import random
from typing import Sequence, TypeVar

Choosable = TypeVar("Choosable")

def choose(items: Sequence[Choosable]) -> Choosable:
    return random.choice(items)

names = ["Guido", "Jukka", "Ivan"]
reveal_type(names)

name = choose(names)
reveal_type(name)

<string>:13: error: Revealed type is 'builtins.list[builtins.str*]'
<string>:16: error: Revealed type is 'builtins.str*'



In [9]:
%%typecheck
# choose.py

import random
from typing import Sequence, TypeVar

Choosable = TypeVar("Choosable", str, float)

def choose(items: Sequence[Choosable]) -> Choosable:
    return random.choice(items)

reveal_type(choose(["Guido", "Jukka", "Ivan"]))
reveal_type(choose([1, 2, 3]))
reveal_type(choose([True, 42, 3.14]))
reveal_type(choose(["Python", 3, 7]))

<string>:12: error: Revealed type is 'builtins.str*'
<string>:13: error: Revealed type is 'builtins.float*'
<string>:14: error: Revealed type is 'builtins.float*'
<string>:15: error: Revealed type is 'builtins.object*'
<string>:15: error: Value of type variable "Choosable" of "choose" cannot be "object"



## The Optional Type

In [3]:

def player_order(names, start=None):
    """Rotate player order so that start goes first"""
    if start is None:
        start = choose(names)
    start_idx = names.index(start)
    return names[start_idx:] + names[:start_idx]

In [6]:
player_order(['apple','bird', 'dog'], 'bird')

['bird', 'dog', 'apple']

In [14]:
%%typecheck
from typing import Sequence, Optional

import random
from typing import Sequence, TypeVar

Choosable = TypeVar("Choosable", str, float)


def choose(items: Sequence[Choosable]) -> Choosable:
    return random.choice(items)

def player_order(
    names: Sequence[str], start: Optional[str] = None
) -> Sequence[str]:
    """Rotate player order so that start goes first"""
    if start is None:
        start = choose(names)
    start_idx = names.index(start)
    return names[start_idx:] + names[:start_idx]

<string>:20: error: Unsupported left operand type for + ("Sequence[str]")



In [16]:
player_order(['apple','bird', 'dog'], 'bird')

['bird', 'dog', 'apple']

## Example: The Object(ive) of the Game

In [17]:
# game.py

import random
import sys

class Card:
    SUITS = "♠ ♡ ♢ ♣".split()
    RANKS = "2 3 4 5 6 7 8 9 10 J Q K A".split()

    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __repr__(self):
        return f"{self.suit}{self.rank}"

class Deck:
    def __init__(self, cards):
        self.cards = cards

    @classmethod
    def create(cls, shuffle=False):
        """Create a new deck of 52 cards"""
        cards = [Card(s, r) for r in Card.RANKS for s in Card.SUITS]
        if shuffle:
            random.shuffle(cards)
        return cls(cards)

    def deal(self, num_hands):
        """Deal the cards in the deck into a number of hands"""
        cls = self.__class__
        return tuple(cls(self.cards[i::num_hands]) for i in range(num_hands))

class Player:
    def __init__(self, name, hand):
        self.name = name
        self.hand = hand

    def play_card(self):
        """Play a card from the player's hand"""
        card = random.choice(self.hand.cards)
        self.hand.cards.remove(card)
        print(f"{self.name}: {card!r:<3}  ", end="")
        return card

class Game:
    def __init__(self, *names):
        """Set up the deck and deal cards to 4 players"""
        deck = Deck.create(shuffle=True)
        self.names = (list(names) + "P1 P2 P3 P4".split())[:4]
        self.hands = {
            n: Player(n, h) for n, h in zip(self.names, deck.deal(4))
        }

    def play(self):
        """Play a card game"""
        start_player = random.choice(self.names)
        turn_order = self.player_order(start=start_player)

        # Play cards from each player's hand until empty
        while self.hands[start_player].hand.cards:
            for name in turn_order:
                self.hands[name].play_card()
            print()

    def player_order(self, start=None):
        """Rotate player order so that start goes first"""
        if start is None:
            start = random.choice(self.names)
        start_idx = self.names.index(start)
        return self.names[start_idx:] + self.names[:start_idx]

# if __name__ == "__main__":
#     # Read player names from command line
#     player_names = sys.argv[1:]
#     game = Game(*player_names)
#     game.play()
player_names = ['apple','bird', 'cat', 'dog']
game = Game(*player_names)
game.play()

dog: ♢Q   apple: ♢6   bird: ♣4   cat: ♢2   
dog: ♠7   apple: ♣Q   bird: ♢5   cat: ♡K   
dog: ♠J   apple: ♠Q   bird: ♣6   cat: ♠2   
dog: ♡8   apple: ♠8   bird: ♡5   cat: ♠6   
dog: ♣A   apple: ♠K   bird: ♠4   cat: ♡J   
dog: ♠3   apple: ♡9   bird: ♣J   cat: ♡4   
dog: ♢A   apple: ♢4   bird: ♠10  cat: ♢7   
dog: ♡A   apple: ♢3   bird: ♢9   cat: ♡10  
dog: ♡3   apple: ♢K   bird: ♣2   cat: ♣7   
dog: ♠5   apple: ♣5   bird: ♢8   cat: ♡Q   
dog: ♣3   apple: ♡6   bird: ♢J   cat: ♡7   
dog: ♢10  apple: ♡2   bird: ♣10  cat: ♠A   
dog: ♣9   apple: ♣K   bird: ♠9   cat: ♣8   
